<a href="https://colab.research.google.com/github/vikash18086/Capstone/blob/master/Capstone_6th_dec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')
!cp '/content/drive/My Drive/IIITD/Capstone/2015_16_Districtwise.csv' .

In [0]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.decomposition import PCA
from numpy import linalg as LA
import numpy.linalg as linalg
import copy
from collections import defaultdict 
from matplotlib import pyplot as plt

from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
import plotly.express as px
from collections import defaultdict 
import pickle

In [0]:
statewise = pd.read_csv('2015_16_Districtwise.csv')

distname = list(statewise['DISTNAME'])
statename = list(statewise['STATNAME'])
statewise = statewise.drop('AC_YEAR', axis=1)
statewise = statewise.drop('STATCD', axis=1)
statewise = statewise.drop('DISTCD', axis=1)
statewise = statewise.drop('DISTNAME', axis=1)
statewise = statewise.drop('DISTRICTS', axis=1)
statewise = statewise.drop('STATNAME', axis=1)
statewise = statewise.drop('BLOCKS', axis=1)
statewise = statewise.drop('VILLAGES', axis=1)
statewise = statewise.drop('CLUSTERS', axis=1)


In [0]:
label = []
for i,j in zip(statename, distname):
    name = i+' - '+j
    label.append(name)

In [0]:
all_features = list(statewise.columns)
statewise  = statewise.fillna(0)

In [0]:
# Normalization of dataset
x = statewise.values.T
min_max_scaler = preprocessing.Normalizer()
x_scaled = min_max_scaler.fit_transform(x)
x_scaled= x_scaled.tolist()
# df = pd.DataFrame(x_scaled)

In [0]:
len(x_scaled[0])

680

### Data visualization and clustering

In [0]:
#TSNE PLOTS
dataset=copy.deepcopy(x_scaled)

X_embedded = TSNE(n_components=2).fit_transform(dataset)

x= [i[0] for i in X_embedded]
y= [i[1] for i in X_embedded]

plt.scatter(x,y)
plt.show()

In [0]:
#Here we find out best k value to perform kmeans clustering.
cost =[] 
for i in range(1, 20): 
    KM = KMeans(n_clusters = i, max_iter = 500) 
    KM.fit(dataset) 
    cost.append(KM.inertia_)  

plt.plot(range(1, 20), cost, color ='g', linewidth ='3') 
plt.xlabel("Value of K") 
plt.ylabel("Sqaured Error (Cost)") 
plt.show()

In [0]:
#APPLY KMEANS AND PLOT CLUSTERS
kmeans = KMeans(n_clusters=15, random_state=0).fit(dataset)  
# kmeans.labels_

knn_labels = list(kmeans.labels_)

cluster_name = defaultdict(list) 
for i,j in zip(knn_labels,label):
    try:
        cluster_name[i].append(j)
    except:
        cluster_name[i] = j

df = pd.DataFrame(list(zip(x, y,knn_labels,label)), columns =['X', 'Y','Label','distname']) 
fig = px.scatter(df, x='X', y='Y', color='Label')
fig.show()

### Feature selection based on variance

In [0]:
dataset = np.array(dataset).T

In [0]:
# Calculating the variance for the features
feature_var=[]
for i in np.array(dataset):
    feature_var.append(np.std(i)**2)

In [0]:
threshold = 0.004
features=[]
for i in range(len(feature_var)):
    if feature_var[i] > threshold:
        features.append(i)

In [0]:
selected_features = []
for i in range(len(all_features)):
    if i in features:
        selected_features.append(all_features[i])

In [0]:
final_dataset = []
for i in selected_features:
    final_dataset.append(list(statewise[i]))

In [0]:
data2 = list(zip(*final_dataset))

In [0]:
df = pd.DataFrame(data2, columns =selected_features) 
df.head()

SCH7P  ENR3G  ENRTOTG  ENR2P  TCH2G  ...  MEC6  CPC6  TOTCLOT1G  TCHRM3  TCHRM4
0      0     23    86180  21139   4093  ...    20     1        117      26       8
1      0    354    89329  46616   3852  ...    36    12        195     130       5
2      0     25    17674  29392   1442  ...     9     1         97     280       0
3      0    544    57684  16809   2691  ...    18     0         40      98       6
4      0      0    28026  12790   1603  ...    23     1         41      63      29

[5 rows x 18 columns]

In [0]:
export_csv = df.to_csv (r'selected_features_5_Dec.csv', index = None, header=True) 

### converting numerical to categorical

In [0]:
statewise = pd.read_csv('selected_features_5_Dec.csv')
statewise.head()

SCH7P  ENR3G  ENRTOTG  ENR2P  TCH2G  ...  MEC6  CPC6  TOTCLOT1G  TCHRM3  TCHRM4
0      0     23    86180  21139   4093  ...    20     1        117      26       8
1      0    354    89329  46616   3852  ...    36    12        195     130       5
2      0     25    17674  29392   1442  ...     9     1         97     280       0
3      0    544    57684  16809   2691  ...    18     0         40      98       6
4      0      0    28026  12790   1603  ...    23     1         41      63      29

[5 rows x 18 columns]

In [0]:
all_features = list(statewise.columns)

In [0]:
data = []
# data.append(statename)
for i in all_features:
    value = list(statewise[i])
    minval = statewise[i].min()
    maxval = statewise[i].max()
    
    split = (maxval - minval)/5
    range1 = split + minval; range2 = range1 + split; range3 = range2 + split; range4 = range3 + split;
    
#     print("minval : ",minval,", maxval : ",maxval)
    col = []
    for i1 in value:    
        if ((i1 >= minval) and (i1 < range1)):
            col.append("low")
        elif ((i1 >=range1) and (i1 < range2)):
            col.append("low-medium")
        elif ((i1 >=range2) and (i1 < range3)):
            col.append("medium")
        elif ((i1 >=range3) and (i1 < range4)):
            col.append("high-medium")
        else:
            col.append("high")
        # if(i1 < range3):
        #   col.append(0)
        # else:
        #   col.append(1)
  
    data.append(col)

In [0]:
data1 = np.array(data)
all_features = list(statewise.columns)
#convert col wise list to row wise list 
data2 = list(zip(*data))

In [0]:
df = pd.DataFrame(data2, columns =all_features) 
df.head()

SCH7P ENR3G ENRTOTG ENR2P       TCH2G  ... MEC6 CPC6 TOTCLOT1G TCHRM3 TCHRM4
0   low   low     low   low  low-medium  ...  low  low       low    low    low
1   low   low     low   low  low-medium  ...  low  low       low    low    low
2   low   low     low   low         low  ...  low  low       low    low    low
3   low   low     low   low  low-medium  ...  low  low       low    low    low
4   low   low     low   low         low  ...  low  low       low    low    low

[5 rows x 18 columns]

In [0]:
export_csv = df.to_csv (r'association_input_5_Dec.csv', index = None, header=True) 

### Read association_input file

In [0]:
statewise = pd.read_csv('association_input_5_Dec.csv')
statewise.head()

SCH7P ENR3G ENRTOTG ENR2P       TCH2G  ... MEC6 CPC6 TOTCLOT1G TCHRM3 TCHRM4
0   low   low     low   low  low-medium  ...  low  low       low    low    low
1   low   low     low   low  low-medium  ...  low  low       low    low    low
2   low   low     low   low         low  ...  low  low       low    low    low
3   low   low     low   low  low-medium  ...  low  low       low    low    low
4   low   low     low   low         low  ...  low  low       low    low    low

[5 rows x 18 columns]

In [0]:
features = statewise.columns
dataset = statewise.values.tolist()
records = dataset

In [0]:
len(records)

680

In [0]:
records = []
for i in range(len(dataset)):
    row = []
    for j in range(len(dataset[0])):
        s = features[j]+'_'+ dataset[i][j]
        row.append(s)
    records.append(row)

In [0]:
 !pip install efficient-apriori

In [0]:
from efficient_apriori import apriori

itemsets, rules = apriori(records, min_support=0.85, min_confidence=1)

In [0]:
rules

[{ENR3G_low} -> {TCHRM3_low},
 {TCHM7_low} -> {SCH7P_low},
 {ENR2P_low, ENR3G_low} -> {TCHRM3_low},
 {ENR2P_low, TCHM7_low} -> {SCH7P_low},
 {ENR3G_low, ENRG3_low} -> {TCHRM3_low},
 {ENR3G_low, PREP_low} -> {TCHRM3_low},
 {ENR3G_low, PRESTD_low} -> {TCHRM3_low},
 {ENR3G_low, SCH7P_low} -> {TCHRM3_low},
 {PREP_low, TCHM7_low} -> {SCH7P_low},
 {TCHM7_low, TCHRM3_low} -> {SCH7P_low},
 {ENR2P_low, ENR3G_low, ENRG3_low} -> {TCHRM3_low},
 {ENR3G_low, ENRG3_low, PREP_low} -> {TCHRM3_low},
 {ENR3G_low, ENRG3_low, SCH7P_low} -> {TCHRM3_low}]

In [0]:
store_rule = []
store_info = [] 
rules_rhs = filter(lambda rule: len(rule.lhs) >= 1 and len(rule.rhs) >= 1, rules)
print("Rules generated By assosiation pattern mining along with confidence, support, lift value are as follows:")
for rule in sorted(rules_rhs, key=lambda rule: rule.lift):
    print(rule)
    conf = rule.confidence 
    supp  = rule.support
    lift = rule.lift
    x = str(str(str(str(rule).split('(')[0]).replace('{','')).replace('}','')).replace(' ->',',')
    y = str(x.replace(' ','')).split(',')
    y_ = []
    for i in y:
        y_.append(i.split('_')[0])
        
    store_info.append([y_,conf,supp,lift])
    store_rule.append(rule)

Rules generated By assosiation pattern mining along with confidence, support, lift value are as follows:
{ENR2P_low, ENR3G_low} -> {TCHRM3_low} (conf: 1.000, supp: 0.879, lift: 1.035, conv: 33823529.412)
{ENR3G_low, PREP_low} -> {TCHRM3_low} (conf: 1.000, supp: 0.876, lift: 1.035, conv: 33823529.412)
{ENR3G_low, PRESTD_low} -> {TCHRM3_low} (conf: 1.000, supp: 0.856, lift: 1.035, conv: 33823529.412)
{ENR3G_low, SCH7P_low} -> {TCHRM3_low} (conf: 1.000, supp: 0.875, lift: 1.035, conv: 33823529.412)
{ENR2P_low, ENR3G_low, ENRG3_low} -> {TCHRM3_low} (conf: 1.000, supp: 0.865, lift: 1.035, conv: 33823529.412)
{ENR3G_low, ENRG3_low, PREP_low} -> {TCHRM3_low} (conf: 1.000, supp: 0.853, lift: 1.035, conv: 33823529.412)
{ENR3G_low, ENRG3_low, SCH7P_low} -> {TCHRM3_low} (conf: 1.000, supp: 0.851, lift: 1.035, conv: 33823529.412)
{ENR3G_low} -> {TCHRM3_low} (conf: 1.000, supp: 0.922, lift: 1.035, conv: 33823529.412)
{ENR3G_low, ENRG3_low} -> {TCHRM3_low} (conf: 1.000, supp: 0.899, lift: 1.035, con

In [0]:
store_info[0]

[['ENR2P', 'ENR3G', 'TCHRM3'], 1.0, 0.8794117647058823, 1.035007610350076]

### Apply Random-Forest Feature selection followed by Apriori

In [0]:
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble.forest import RandomForestClassifier

In [0]:
statewise = pd.read_csv('2015_16_Districtwise.csv')

distname = list(statewise['DISTNAME'])
statename = list(statewise['STATNAME'])
statewise = statewise.drop('AC_YEAR', axis=1)
statewise = statewise.drop('STATCD', axis=1)
statewise = statewise.drop('DISTCD', axis=1)
statewise = statewise.drop('DISTNAME', axis=1)
statewise = statewise.drop('DISTRICTS', axis=1)
statewise = statewise.drop('STATNAME', axis=1)
statewise = statewise.drop('BLOCKS', axis=1)
statewise = statewise.drop('VILLAGES', axis=1)
statewise = statewise.drop('CLUSTERS', axis=1)

label = []
for i,j in zip(statename, distname):
    name = i+' - '+j
    label.append(name)

all_features = list(statewise.columns)
statewise  = statewise.fillna(0)

In [0]:
feature_label = statewise.columns

In [0]:
statewise.head()

TOTPOPULAT  P_URB_POP  POPULATION_0_6  ...  TOTCLS5G  TOTCLS6G  TOTCLS7G
0    875564.0      11.33       197001.90  ...        23       852       239
1   1015503.0      17.19       161871.18  ...        30      1665       360
2   1269751.0      98.73       155798.45  ...         0      2873        40
3    735753.0      11.74       150755.79  ...         8      1035       325
4    570060.0      13.80        97651.28  ...         5      1101       209

[5 rows x 810 columns]

In [0]:
X = np.array(statewise)
Y = label
sel = SelectFromModel(RandomForestClassifier(n_estimators = 100)).fit(X,Y)

In [0]:
#return boolean value corresponding to index of selected features
sel_feat = list(sel.get_support())
#select features return by Random forest 
selected_features = []
for i,j in zip(sel_feat,all_features):
  if i:
    selected_features.append(j)

In [0]:
if len(selected_features) > 30:
  selected_features = selected_features[:30]

In [0]:
final_dataset = []
for i in selected_features:
    final_dataset.append(list(statewise[i]))

data2 = list(zip(*final_dataset))

df = pd.DataFrame(data2, columns =selected_features)
export_csv = df.to_csv (r'selected_features_5_Dec.csv', index = None, header=True) 
df.head()

P_URB_POP  POPULATION_0_6  GROWTHRATE  SEXRATIO  ...  ENR5   ENR6  ENRTOT  ENR4G
0      11.33       197001.90       34.62     843.0  ...   206  16030  127558    545
1      17.19       161871.18       20.34     873.0  ...   244  36272  183107    348
2      98.73       155798.45       23.56     879.0  ...     0  50880  119458     44
3      11.74       150755.79       21.18     883.0  ...    71  17407   95521    488
4      13.80        97651.28       29.18     913.0  ...    79  18741   61345    517

[5 rows x 30 columns]

### converting numerical to categorical

In [0]:
statewise = pd.read_csv('selected_features_5_Dec.csv')
statewise.head()

all_features = list(statewise.columns)

data = []
# data.append(statename)
for i in all_features:
    value = list(statewise[i])
    minval = statewise[i].min()
    maxval = statewise[i].max()
    
    split = (maxval - minval)/5
    range1 = split + minval; range2 = range1 + split; range3 = range2 + split; range4 = range3 + split;
    
#     print("minval : ",minval,", maxval : ",maxval)
    col = []
    for i1 in value:    
        if ((i1 >= minval) and (i1 < range1)):
            col.append("low")
        elif ((i1 >=range1) and (i1 < range2)):
            col.append("low-medium")
        elif ((i1 >=range2) and (i1 < range3)):
            col.append("medium")
        elif ((i1 >=range3) and (i1 < range4)):
            col.append("high-medium")
        else:
            col.append("high")
        # if(i1 < range3):
        #   col.append(0)
        # else:
        #   col.append(1)
  
    data.append(col)

data1 = np.array(data)
all_features = list(statewise.columns)
#convert col wise list to row wise list 
data2 = list(zip(*data))

df = pd.DataFrame(data2, columns =all_features) 
df.head()

P_URB_POP POPULATION_0_6 GROWTHRATE     SEXRATIO  ... ENR5 ENR6 ENRTOT ENR4G
0  low-medium            low     medium  high-medium  ...  low  low    low   low
1  low-medium            low     medium  high-medium  ...  low  low    low   low
2        high            low     medium  high-medium  ...  low  low    low   low
3  low-medium            low     medium  high-medium  ...  low  low    low   low
4  low-medium            low     medium  high-medium  ...  low  low    low   low

[5 rows x 30 columns]

In [0]:
#update df
df12 = np.array(df) 
feature_name = df.columns
for i in range(len(df12)):
  for j in range(len(df12[i])):
    df12[i][j] = feature_name[j]+'_'+df12[i][j]
df = pd.DataFrame(df12, columns =feature_name) 

In [0]:
export_csv = df.to_csv (r'association_input_5_Dec.csv', index = None, header=True) 

### Read association_input file

In [0]:
statewise = pd.read_csv('association_input_5_Dec.csv')
statewise.head()

P_URB_POP      POPULATION_0_6  ...      ENRTOT      ENR4G
0  P_URB_POP_low-medium  POPULATION_0_6_low  ...  ENRTOT_low  ENR4G_low
1  P_URB_POP_low-medium  POPULATION_0_6_low  ...  ENRTOT_low  ENR4G_low
2        P_URB_POP_high  POPULATION_0_6_low  ...  ENRTOT_low  ENR4G_low
3  P_URB_POP_low-medium  POPULATION_0_6_low  ...  ENRTOT_low  ENR4G_low
4  P_URB_POP_low-medium  POPULATION_0_6_low  ...  ENRTOT_low  ENR4G_low

[5 rows x 30 columns]

In [0]:
features = statewise.columns
dataset = statewise.values.tolist()
records = dataset

# records = []
# for i in range(len(dataset)):
#     row = []
#     for j in range(len(dataset[0])):
#         s = features[j]+'_'+ dataset[i][j]
#         row.append(s)
#     records.append(row)

In [0]:
itemsets, rules = apriori(records, min_support=0.85, min_confidence=1)

In [0]:
len(rules)

23

In [0]:
store_rule = []
store_info = [] 
rules_rhs = filter(lambda rule: len(rule.lhs) >= 1 and len(rule.rhs) >= 1, rules)
print("Rules generated By assosiation pattern mining along with confidence, support, lift value are as follows:")
for rule in sorted(rules_rhs, key=lambda rule: rule.lift):
    print(rule)
    conf = rule.confidence 
    supp  = rule.support
    lift = rule.lift
    x = str(str(str(str(rule).split('(')[0]).replace('{','')).replace('}','')).replace(' ->',',')
    y = str(x.replace(' ','')).split(',')
    y_ = []
    for i in y:
        y_.append(i.split('_')[0])
        
    store_info.append([y_,conf,supp,lift])
    store_rule.append(rule)

Rules generated By assosiation pattern mining along with confidence, support, lift value are as follows:
{ENR3_low, SCH6G_low} -> {SCHGIR3_low} (conf: 1.000, supp: 0.887, lift: 1.037, conv: 35294117.647)
{ENR3_low, ENR6_low, SCH6G_low} -> {SCHGIR3_low} (conf: 1.000, supp: 0.874, lift: 1.037, conv: 35294117.647)
{SCH7_low} -> {SCHGIR7_low} (conf: 1.000, supp: 0.918, lift: 1.086, conv: 79411764.706)
{AREA_SQKM_low, SCH7_low} -> {SCHGIR7_low} (conf: 1.000, supp: 0.853, lift: 1.086, conv: 79411764.706)
{ENR3_low, SCH7_low} -> {SCHGIR7_low} (conf: 1.000, supp: 0.850, lift: 1.086, conv: 79411764.706)
{ENR6_low, SCH7_low} -> {SCHGIR7_low} (conf: 1.000, supp: 0.906, lift: 1.086, conv: 79411764.706)
{SCH6G_low, SCH7_low} -> {SCHGIR7_low} (conf: 1.000, supp: 0.853, lift: 1.086, conv: 79411764.706)
{ENR5_low, ENR6_low, SCH7_low} -> {SCHGIR7_low} (conf: 1.000, supp: 0.884, lift: 1.086, conv: 79411764.706)
{ENR5_low, SCH5GR_low, SCH7_low} -> {SCHGIR7_low} (conf: 1.000, supp: 0.860, lift: 1.086, con

In [0]:
store_info[0]

[['ENR3', 'SCH6G', 'SCHGIR3'], 1.0, 0.8867647058823529, 1.0365853658536586]

### Pickle dump & load

In [0]:
# pickle dump
itemsets_file = open("itemsets","wb")
pickle.dump(itemsets,itemsets_file)
itemsets_file.close()

rules_file = open("rules","wb")
pickle.dump(rules,rules_file)
rules_file.close()

In [0]:
#pickle load
itemsets_file = open("itemsets",'rb')
itemsets = pickle.load(itemsets_file)
itemsets_file.close()

rules_file = open("rules",'rb')
rules = pickle.load(rules_file)
rules_file.close()